In [1]:
import sys
import os

from os import listdir
from os.path import isfile, join
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(root))

In [2]:
import pandas as pd

from datetime import datetime, timedelta

### 데이터 원본

In [16]:
status_path = join(root, 'data', 'extracted', 'Korea_Covid_Patient.csv')
status = pd.read_csv(status_path)
status.head()

,District,State,data_date,new_pat,pop,long,lat,no_pat
0,Seoul,Seoul,20200122,0,9689159,126.978,37.5665,0
1,Seoul,Seoul,20200123,0,9689159,126.978,37.5665,0
2,Seoul,Seoul,20200124,1,9689159,126.978,37.5665,1
3,Seoul,Seoul,20200125,0,9689159,126.978,37.5665,1
4,Seoul,Seoul,20200126,0,9689159,126.978,37.5665,1


In [17]:
route_path = join(root, 'data', 'extracted', 'merged_route_final.csv')
route = pd.read_csv(route_path)
route['date'] = pd.to_datetime(route['date'])
route.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,2000000001,gangnam,overseas inflow,2020-01-22,hospital,37.524355,127.027948,196.0,169.0
1,1400000003,yongsan,etc,2020-01-23,etc,37.536606,126.977140,182.0,131.0
2,2000000001,gangnam,overseas inflow,2020-01-23,store,37.527752,127.019480,192.0,163.0
3,1000000001,jung,overseas inflow,2020-01-24,hospital,37.567241,127.005659,147.0,152.0
4,1400000003,jongno,etc,2020-01-24,etc,37.579617,126.977041,133.0,131.0


In [18]:
# type, reason column type에 맞게 수정

type_shrink_path = join(root, 'data', 'checklist', 'type_column.csv')
type_shrink_df = pd.read_csv(type_shrink_path)

reason_shrink_path = join(root, 'data', 'checklist', 'infection_case_column.csv')
reason_shrink_df = pd.read_csv(reason_shrink_path)

for index, row in route.iterrows():
    new_type = type_shrink_df.loc[type_shrink_df['type'] == row['type']].iloc[0]['type_1']
    route.loc[index, 'type'] = new_type
    
    new_reason = reason_shrink_df.loc[reason_shrink_df['infection_case'] == row['infection_case']].iloc[0]['infection_case_1']
    route.loc[index, 'infection_case'] = new_reason
    
route.head()

,patient_id,city,infection_case,date,type,latitude,longitude,row,col
0,2000000001,gangnam,overseas,2020-01-22,medical_institution,37.524355,127.027948,196.0,169.0
1,1400000003,yongsan,other,2020-01-23,other,37.536606,126.977140,182.0,131.0
2,2000000001,gangnam,overseas,2020-01-23,store,37.527752,127.019480,192.0,163.0
3,1000000001,jung,overseas,2020-01-24,medical_institution,37.567241,127.005659,147.0,152.0
4,1400000003,jongno,other,2020-01-24,other,37.579617,126.977041,133.0,131.0


### 방문지 유형 상관 관계

In [19]:
types = list(dict.fromkeys(route['type'].tolist()))
types.insert(0, 'date')

type_df = pd.DataFrame(columns=types)
for index1, day_status in status.iterrows():
    date = datetime.strptime(str(day_status['data_date']), '%Y%m%d')
    type_df = type_df.append({'date': date}, ignore_index=True)
    type_df = type_df.fillna(0)
    
    rows = route.loc[route['date'] == date]
    for index2, row in rows.iterrows():
        type_index = type_df.loc[type_df['date'] == date].index[0]
        type_df.loc[type_index, row['type']] += 1

type_df.head()

,date,medical_institution,other,store,fnb,public_transportation,religious_facility,entertainment_facility,service_facility,sports_facility,...,club,public_institution,finantial_institution,gethering,outdoor,sauna,work,door_sales,exhibition,hall
0,2020-01-22,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,0,2,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,0,1,2,1,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
type_path = join(root, 'data', 'checklist', 'type_cnt.csv')
type_df.to_csv(type_path, encoding='utf-8-sig', index=False)

In [21]:
type_df = type_df.set_index(['date'])
type_df.insert (0, 'new_patients', 0)

for index, row in type_df.iterrows():
    str_date = index.strftime('%Y%m%d')
    row['new_patients'] = status.loc[status['data_date'] == int(str_date)].iloc[0]['new_pat']
    
type_df.head()

,new_patients,medical_institution,other,store,fnb,public_transportation,religious_facility,entertainment_facility,service_facility,sports_facility,...,club,public_institution,finantial_institution,gethering,outdoor,sauna,work,door_sales,exhibition,hall
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-25,0,0,2,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-26,0,0,1,2,1,2,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
sorted_types = type_df.corr().iloc[:,:1].T.iloc[0].sort_values(ascending=False).to_dict()
new_dict = dict(filter(lambda elem: elem[1] > 0.3, sorted_types.items()))
new_list = list(new_dict.keys())
print(new_list)
print()

['new_patients', 'educational_institution', 'hall', 'store', 'finantial_institution', 'sports_facility', 'fnb', 'service_facility']



In [23]:
corr_path = join(root, 'data', 'checklist', 'type_corr.csv')
type_df.corr().to_csv(corr_path)

In [29]:
type_df.corr()

,new_patients,medical_institution,other,store,fnb,public_transportation,religious_facility,entertainment_facility,service_facility,sports_facility,...,club,public_institution,finantial_institution,gethering,outdoor,sauna,work,door_sales,exhibition,hall
new_patients,1.000000,0.250715,-0.145466,0.398482,0.325912,0.034169,0.109536,-0.064555,0.317406,0.347747,...,-0.052270,0.036585,0.379207,-0.010854,0.041045,0.052375,0.129594,-0.029201,0.109516,0.404484
medical_institution,0.250715,1.000000,0.476572,0.379891,0.200454,0.301724,0.143266,0.075029,0.260077,0.074955,...,-0.075861,0.016640,0.151241,-0.022533,-0.026846,-0.039873,0.166411,0.042394,0.008824,0.177308
other,-0.145466,0.476572,1.000000,0.213139,0.205182,0.319413,0.300489,0.244656,0.117413,0.033827,...,-0.030645,0.080370,-0.046058,-0.030355,0.043309,0.082029,0.051413,0.216484,-0.045460,-0.067895
store,0.398482,0.379891,0.213139,1.000000,0.594370,0.334859,0.228120,0.246400,0.273961,0.179856,...,-0.046553,0.120163,0.286853,0.314482,0.057712,0.139400,0.481274,-0.015424,0.103841,0.249844
fnb,0.325912,0.200454,0.205182,0.594370,1.000000,0.377020,0.222978,0.482926,0.342457,0.303337,...,0.333193,0.189205,0.244943,0.216588,0.194646,0.074456,0.198393,0.063382,0.154772,0.240051
public_transportation,0.034169,0.301724,0.319413,0.334859,0.377020,1.000000,0.114225,0.272226,0.187135,0.020999,...,-0.009875,0.391238,0.002188,0.041125,0.136826,-0.007404,0.209127,0.021787,0.178734,0.080636
religious_facility,0.109536,0.143266,0.300489,0.228120,0.222978,0.114225,1.000000,0.093224,0.162691,0.102329,...,-0.040815,0.192336,-0.009287,-0.051860,0.126477,0.245845,0.262963,0.024082,-0.022738,0.009463
entertainment_facility,-0.064555,0.075029,0.244656,0.246400,0.482926,0.272226,0.093224,1.000000,0.075572,0.163438,...,0.061708,0.240684,0.040440,0.122211,0.137444,0.149303,0.014523,0.183411,-0.030412,0.054632
service_facility,0.317406,0.260077,0.117413,0.273961,0.342457,0.187135,0.162691,0.075572,1.000000,0.455702,...,-0.038453,0.120936,0.412058,-0.006577,0.102981,0.027598,0.229522,0.054023,0.066474,0.383336
sports_facility,0.347747,0.074955,0.033827,0.179856,0.303337,0.020999,0.102329,0.163438,0.455702,1.000000,...,0.033366,0.077944,0.392574,-0.025597,-0.024055,0.069190,0.242566,0.394245,-0.007332,0.385282


### 감염 원인 상관 관계

In [24]:
reasons = list(dict.fromkeys(route['infection_case'].tolist()))
reasons.insert(0, 'date')

reason_df = pd.DataFrame(columns=reasons)
for index1, day_status in status.iterrows():
    date = datetime.strptime(str(day_status['data_date']), '%Y%m%d')
    reason_df = reason_df.append({'date': date}, ignore_index=True)
    reason_df = reason_df.fillna(0)
    
    rows = route.loc[route['date'] == date]
    for index2, row in rows.iterrows():
        reason_index = reason_df.loc[reason_df['date'] == date].index[0]
        reason_df.loc[reason_index, row['infection_case']] += 1

reason_df.head()

,date,overseas,other,contact,hospital,building,church,call_center,health_facility,educational_facility,...,network_marketing,nursing_facility,company,gethering,public_transportation,office,bank,protest,theater,cafe
0,2020-01-22,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,4,1,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
reason_path = join(root, 'data', 'checklist', 'reason_cnt.csv')
reason_df.to_csv(reason_path, encoding='utf-8-sig', index=False)

In [26]:
reason_df = reason_df.set_index(['date'])
reason_df.insert (0, 'new_patients', 0)

for index, row in reason_df.iterrows():
    str_date = index.strftime('%Y%m%d')
    row['new_patients'] = status.loc[status['data_date'] == int(str_date)].iloc[0]['new_pat']
    
reason_df.head()

,new_patients,overseas,other,contact,hospital,building,church,call_center,health_facility,educational_facility,...,network_marketing,nursing_facility,company,gethering,public_transportation,office,bank,protest,theater,cafe
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-25,0,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-26,0,4,1,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
reason_df.corr()

,new_patients,overseas,other,contact,hospital,building,church,call_center,health_facility,educational_facility,...,network_marketing,nursing_facility,company,gethering,public_transportation,office,bank,protest,theater,cafe
new_patients,1.000000,0.138834,0.588314,0.233087,-0.109059,-0.082045,0.530759,-0.056435,-0.024863,0.097242,...,-0.010677,0.039720,-0.008347,0.279860,-0.002073,0.107594,-0.025148,0.261907,0.059263,0.224172
overseas,0.138834,1.000000,0.351121,0.338501,-0.047415,-0.047755,-0.068819,0.056465,-0.088648,-0.080021,...,-0.101864,-0.070123,-0.047547,-0.096060,-0.067627,-0.076998,-0.039243,-0.035007,-0.051439,-0.031994
other,0.588314,0.351121,1.000000,0.478608,0.211011,0.197594,0.547081,0.061795,-0.071024,0.232697,...,-0.124979,-0.046658,-0.065895,0.175916,-0.076788,-0.015206,-0.059546,0.368017,0.314198,0.147354
contact,0.233087,0.338501,0.478608,1.000000,-0.024476,0.144800,0.389558,0.404458,-0.075904,0.044527,...,-0.107192,0.048988,-0.016172,0.144388,0.011783,0.040273,0.076549,0.161684,0.275037,0.054038
hospital,-0.109059,-0.047415,0.211011,-0.024476,1.000000,0.210832,-0.003114,-0.044401,-0.039473,-0.044522,...,-0.060617,-0.041878,-0.031903,-0.069137,-0.037177,-0.058609,-0.028829,-0.044370,-0.031127,-0.017588
building,-0.082045,-0.047755,0.197594,0.144800,0.210832,1.000000,0.054394,0.017203,-0.029571,-0.031961,...,-0.051915,-0.035866,-0.027323,-0.059211,-0.031840,0.003937,-0.024690,-0.038000,-0.026658,-0.015063
church,0.530759,-0.068819,0.547081,0.389558,-0.003114,0.054394,1.000000,0.071894,0.040923,0.273816,...,-0.021954,-0.009312,-0.022582,0.330019,-0.048572,0.056946,0.043072,0.432346,0.101107,0.004446
call_center,-0.056435,0.056465,0.061795,0.404458,-0.044401,0.017203,0.071894,1.000000,-0.040061,-0.033190,...,-0.042774,-0.033875,-0.022672,-0.051850,-0.030073,-0.047410,-0.023320,-0.023790,-0.018402,-0.014227
health_facility,-0.024863,-0.088648,-0.071024,-0.075904,-0.039473,-0.029571,0.040923,-0.040061,1.000000,0.018962,...,0.623989,0.150051,0.386276,-0.052285,-0.028115,-0.044323,-0.021802,-0.006529,0.040990,-0.013301
educational_facility,0.097242,-0.080021,0.232697,0.044527,-0.044522,-0.031961,0.273816,-0.033190,0.018962,1.000000,...,-0.033279,-0.029374,-0.022377,0.029548,-0.026077,-0.041110,-0.020221,0.346812,0.514848,-0.012337


In [34]:
sorted_types = reason_df.corr().iloc[:,:1].T.iloc[0].sort_values(ascending=False).to_dict()
sorted_types
# new_dict = dict(filter(lambda elem: elem[1] > 0.3, sorted_types.items()))
# new_list = list(new_dict.keys())
# print(new_list)
# print()

{'new_patients': 1.0,
 'other': 0.5883138693614303,
 'church': 0.5307593804920245,
 'gethering': 0.2798600903553677,
 'protest': 0.26190666627549064,
 'contact': 0.2330868169170341,
 'cafe': 0.22417156492703127,
 'overseas': 0.13883439300109698,
 'office': 0.10759418605527607,
 'educational_facility': 0.09724217996271332,
 'theater': 0.05926289641831317,
 'nursing_facility': 0.0397199097010172,
 'public_transportation': -0.002073372272212923,
 'public_institution': -0.0034106841033013815,
 'company': -0.008346679512418041,
 'network_marketing': -0.01067687749625348,
 'health_facility': -0.024862652310569996,
 'bank': -0.025147910340661595,
 'insurance': -0.03369260331722541,
 'family': -0.05179314764116592,
 'call_center': -0.0564346642820208,
 'logistics_center': -0.05736835316596647,
 'building': -0.08204518766299529,
 'club': -0.09368236876137617,
 'hospital': -0.1090589460883626}

In [28]:
corr_path = join(root, 'data', 'checklist', 'reason_corr.csv')
reason_df.corr().to_csv(corr_path)